<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"
alt="Rubin Observatory logo, a graphical representation of turning stars into data."> <br><br>
**Description:** Generate parameters for the _Rubin Observatory DP1_ paper. <br>
**Authors:** Leanne Guy <br>
**Last verified to run:** 2025-05-14 <br>
**LSST Science Pipelines version:** v29.0.0<br>
**Container size:** medium <br>

# DP1 parameters

## Setup

In [47]:
! pip install inflect > /dev/null 2>&1

In [45]:
# Generic python packages
import os
import csv
import requests
import yaml
import numpy as np
import pandas as pd
import numpy as np
from pathlib import Path


# LSST Science Pipelines 
from lsst.daf.butler import Butler

# RSP
from lsst.rsp import get_tap_service

import warnings
warnings.filterwarnings("ignore")

### Define utility functions

In [ ]:
# Custom formatter: no trailing zeros if not needed
def custom_float(x):
    if isinstance(x, float):
        s = f"{x:.3f}" #.rstrip('0').rstrip('.')
        return s
    return x

In [ ]:
# Function to round to N significant figures
def round_sf(x, sig=3):
    return np.round(x, sig - int(np.floor(np.log10(abs(x)))) - 1)

In [56]:
# Template for a parameter
def format_param(key, value):
    return f"\\newcommand{{\\{key}}}{{{value}\\xspace}}"

### Set up DP1 Butler

In [29]:
instrument = 'LSSTComCam'
collections = ['LSSTComCam/DP1/defaults', 
               'LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260',
               'skymaps', ]
skymap = 'lsst_cells_v1'
butler = Butler("/repo/dp1",
                instrument=instrument, 
                collections=collections, 
                skymap=skymap)
registry = butler.registry
skymap = butler.get('skyMap', skymap=skymap)

In [1]:
## Consdb
# Use the consdb TAP_ service 
from lsst.rsp import get_tap_service

cdb = get_tap_service("consdbtap")
assert cdb is not None

In [2]:
# Look at the cdb_lsstcomcam schema
query = "SELECT * FROM tap_schema.tables " \
        "WHERE tap_schema.tables.schema_name = 'cdb_lsstcomcam'" \
        "ORDER BY table_index ASC"
tables = cdb.search(query).to_table()
tables

description,schema_name,table_index,table_name,table_type,utype
str512,str512,int32,str64,str8,str512
Information from observatory systems about each detector (CCD) within each raw image taken,cdb_lsstcomcam,0,cdb_lsstcomcam.ccdexposure,table,
Information from the Camera Control System about each detector (CCD) within each raw image taken,cdb_lsstcomcam,0,cdb_lsstcomcam.ccdexposure_camera,table,
Flexible key/value metadata about each detector (CCD) within each raw image taken; used for development and engineering purposes,cdb_lsstcomcam,0,cdb_lsstcomcam.ccdexposure_flexdata,table,
Key names and value types used in the ccdexposure_flexdata table,cdb_lsstcomcam,0,cdb_lsstcomcam.ccdexposure_flexdata_schema,table,
Information from Summit Rapid Analysis about each detector (CCD) within each raw image taken,cdb_lsstcomcam,0,cdb_lsstcomcam.ccdexposure_quicklook,table,
Information from observatory systems about each detector (CCD) within each visit in visit system 1 (visit per exposure); describes detectors in processed visit images (or calibrated exposures),cdb_lsstcomcam,0,cdb_lsstcomcam.ccdvisit1,table,
Information from Summit Rapid Analysis about each detector (CCD) within each visit in visit system 1 (visit per exposure); describes detectors in processed visit images (or calibrated exposures),cdb_lsstcomcam,0,cdb_lsstcomcam.ccdvisit1_quicklook,table,
Information from observatory systems about each raw image taken,cdb_lsstcomcam,0,cdb_lsstcomcam.exposure,table,
Flexible key/value metadata about each raw image taken; used for development and engineering purposes,cdb_lsstcomcam,0,cdb_lsstcomcam.exposure_flexdata,table,


# Compute parameters

In [50]:
# params dictionary
params = {}

In [53]:
# Extract filter and pointing information about the fileds from the DP1 exposures in the Butler
exposures = registry.queryDimensionRecords('exposure')
exp_df = pd.DataFrame(columns=['id', 'target', 'physical_filter','ra', 'dec'])
for count, info in enumerate(exposures):
    try:
        exp_df.loc[count] = [info.id, info.target_name, info.physical_filter, 
                         info.tracking_ra, info.tracking_dec,]
    except: 
        print(">>>   Unexpected error:", sys.exc_info()[0])

# slew_icrs is the ECDFS field -- combine them
exp_df.loc[exp_df['target'] == 'slew_icrs', 'target'] = 'ECDFS'
dp1_fields = exp_df.target.unique()

params["nfields"] = len(dp1_fields)

In [58]:
parameters_filepath = str(Path('../parameters.tex').resolve())
with open(parameters_filepath, "w") as file:
    for key, value in params.items():
        formatted_line = format_param(key, value)
        file.write(formatted_line + "\n")

In [ ]:
# Close parameters file
params.close()